In [1]:

import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg

import keras as k
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [2]:
%cd 'E:\Stevens\Spring 2021\Research\Stevens_AM_lab'
%pwd

[WinError 123] The filename, directory name, or volume label syntax is incorrect: "'E:\\Stevens\\Spring 2021\\Research\\Stevens_AM_lab'"
C:\Users\Dalli\Desktop\jiaqi


'C:\\Users\\Dalli\\Desktop\\jiaqi'

In [3]:
import time

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)

In [21]:
from random import shuffle
def create_dataset(img_folder,IMG_HEIGHT=30, IMG_WIDTH=30):
    tic()
    i=1
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        print(dir1)
        files = os.listdir(os.path.join(img_folder, dir1))
        shuffle(files)
        i=0
        for file in files:
          if file.endswith('.png'):
            if (i%1000 == 0):
              print(i)
              toc()
              tic()
            #if (i == 2000):
              #break
            i = i+1
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name

# extract the image array and class name
img_folder=r'E:\Stevens\Spring 2021\Research\Stevens_AM_lab\Test_data_labeled\labeled'
img_data, class_name =create_dataset(img_folder)
toc()

Empty
0
Elapsed time: 0.310009 seconds.

1000
Elapsed time: 2.451142 seconds.

2000
Elapsed time: 1.691430 seconds.

3000
Elapsed time: 1.662099 seconds.

OK
0
Elapsed time: 0.270312 seconds.

1000
Elapsed time: 11.049730 seconds.

2000
Elapsed time: 11.683376 seconds.

3000
Elapsed time: 11.675758 seconds.

4000
Elapsed time: 20.722862 seconds.

5000
Elapsed time: 14.025158 seconds.

6000
Elapsed time: 13.760969 seconds.

7000
Elapsed time: 18.550912 seconds.

8000
Elapsed time: 14.182205 seconds.

9000
Elapsed time: 17.195850 seconds.

10000
Elapsed time: 14.488771 seconds.

11000
Elapsed time: 13.857116 seconds.

12000
Elapsed time: 14.245243 seconds.

13000
Elapsed time: 12.714983 seconds.

Over
0
Elapsed time: 4.512597 seconds.

1000
Elapsed time: 2.908859 seconds.

2000
Elapsed time: 2.869027 seconds.

3000
Elapsed time: 3.225779 seconds.

4000
Elapsed time: 3.131212 seconds.

5000
Elapsed time: 3.582546 seconds.

6000
Elapsed time: 3.861322 seconds.

7000
Elapsed time: 3.812706 

In [5]:

!pip install --upgrade scipy

In [22]:

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = class_name
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[1001, :])])
print(inverted)

['Empty' 'Empty' 'Empty' ... 'Under' 'Under' 'Under']
[0 0 0 ... 3 3 3]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
['Empty']


In [7]:
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[999, :])])
print(inverted)

['Empty']


In [8]:
class Same_net(tf.keras.Model):
    def __init__(self,
                     filters_1x1=16,
                     filters_3x3_reduce=16,
                     filters_3x3=16,
                     filters_5x5_reduce=16,
                     filters_5x5=16,
                     filters_pool_proj=16,
                     name='inception_3a',
                     inp_shapes = (30, 30, 3)):
      
        super(Same_net, self).__init__()
        self.conv1x1 = tf.keras.layers.Conv2D(filters_1x1, (1, 1), padding='same')
        
        self.conv3x3_reduce = tf.keras.layers.Conv2D(filters_3x3_reduce, (1, 1), padding='same')
        self.conv3x3 = tf.keras.layers.Conv2D(filters_3x3, (3, 3), padding='same')
        
        self.conv5x5_reduce = tf.keras.layers.Conv2D(filters_5x5_reduce, (1, 1), padding='same')
        self.conv5x5 = tf.keras.layers.Conv2D(filters_5x5, (5, 5), padding='same')
        
        self.convpool = tf.keras.layers.Conv2D(filters_1x1, (1, 1), padding='same')
        
        self.bn = tf.keras.layers.BatchNormalization()
        self.act = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((3, 3),strides=(1, 1), padding='same')
        self.Concat = tf.keras.layers.Concatenate()

    def call(self, inputs):
        
        b1 = self.conv1x1(inputs)
        b1_act = self.act(b1)
        
        b2 = self.conv3x3_reduce(inputs)
        b2_act = self.act(b2)
        b2_2 = self.conv3x3(b2_act)
        b2_2_act = self.act(b2_2)
        
        b3 = self.conv5x5_reduce(inputs)
        b3_act = self.act(b3)
        b3_2 = self.conv5x5(b3_act)
        b3_2_act = self.act(b3_2)
        
        b4 = self.max_pool(inputs)
        b4_2 = self.convpool(b4)
        b4_2_act = self.act(b4_2)
               
        output = self.Concat([b1_act, b2_2_act, b3_2_act, b4_2_act])
        return output

In [9]:
class Encod_2(tf.keras.Model):
    def __init__(self,filters_1x1=16):
        super(Encod_2, self).__init__()
        self.conv1x1 = tf.keras.layers.Conv2D(filters_1x1, (3, 3), padding='same')
        self.conv1x1_2 = tf.keras.layers.Conv2D(filters_1x1, (3, 3), padding='same')
        self.max_pool2 = tf.keras.layers.MaxPool2D((2,2),strides=(2, 2), padding='same')
        self.act = tf.keras.layers.Activation('relu')
    def call(self, inputs):
        b1 = self.conv1x1(inputs)
        b1_act = self.act(b1)  
        b2 = self.conv1x1_2(b1_act)
        b2_act = self.act(b2)  
        b3 =   self.max_pool2(b2_act) 
        return b3  

In [10]:
class Encod_comp(tf.keras.Model):
    def __init__(self,filters_1x1=16):
        super(Encod_comp, self).__init__()
        self.b1 = Same_net()
        self.b12 = Same_net()
        self.b13 = Same_net()
        
        self.b2 = Encod_2()
        self.b22 = Encod_2()
        self.b23 = Encod_2()

    def call(self, inputs):
        l1 = self.b1(inputs)
        l2 = self.b2(l1)
        l3 = self.b12(l2)
        l4 = self.b22(l3)
        l5 = self.b13(l4)
        #l6 = self.b23(l5)

               
        return l5

In [11]:
class Class_out(tf.keras.Model):
    def __init__(self,filters_1x1=8):
        super(Class_out, self).__init__()
        self.Flat = tf.keras.layers.Flatten()
        self.D1 = tf.keras.layers.Dense(150, activation='relu')
        self.D2 = tf.keras.layers.Dense(40, activation='relu')
        self.D3 = tf.keras.layers.Dense(12, activation='relu')
        self.D4 = tf.keras.layers.Dense(4, activation='softmax')
        #self.D5 = tf.keras.layers.Softmax()
        #self.D0 = Encod_comp()

    def call(self, inputs):
      
        #l0 = self.D0(inputs)
        #l1 = self.Flat(l0)

        l1 = self.Flat(inputs)
        l2 = self.D1(l1)
        l3 = self.D2(l2)
        l4 = self.D3(l3)
        l5 = self.D4(l4)
        #l6 = self.D5(l5)
               
        return l5

In [12]:
class Decoder(tf.keras.Model):
    def __init__(self,filters_1x1=16,same_pad = 0,up_sample=0):
        super(Decoder, self).__init__()
        if (same_pad == 1):
          self.conv1x1 = tf.keras.layers.Conv2DTranspose(filters_1x1, (3, 3), padding='same')
          self.conv1x1_2 = tf.keras.layers.Conv2DTranspose(filters_1x1, (3, 3), padding='same')
        elif (same_pad == 0):
          self.conv1x1 = tf.keras.layers.Conv2DTranspose(filters_1x1, (3, 3), padding='valid')
          self.conv1x1_2 = tf.keras.layers.Conv2DTranspose(filters_1x1, (3, 3), padding='valid')
        else:
           self.conv1x1 = tf.keras.layers.Conv2D(filters_1x1, (3, 3))
           self.conv1x1_2 = tf.keras.layers.Conv2DTranspose(filters_1x1, (3, 3), padding='same')
        if up_sample :
          self.UpSampling2D = tf.keras.layers.UpSampling2D((2, 2), interpolation='bilinear')
        else:
          self.UpSampling2D = tf.keras.layers.UpSampling2D((1, 1), interpolation='bilinear')
        self.act = tf.keras.layers.Activation('sigmoid')

        self.conv_reduce = tf.keras.layers.Conv2D(filters_1x1, (3, 3))

    def call(self, inputs):
        b1 = self.conv1x1(inputs)
        b1_act = self.act(b1)  
        b2 = self.conv1x1_2(b1_act)
        b2_act = self.act(b2)  
        b3 =   self.UpSampling2D(b2_act) 
        return b3  

In [13]:
class Decod_comp(tf.keras.Model):
    def __init__(self,filters_1x1=8):
        super(Decod_comp, self).__init__()
        self.b1 = Same_net()
        self.b12 = Same_net()
        self.b13 = Same_net()
        self.b14 = Same_net()
        
        self.b2 = Decoder(same_pad = 1,up_sample=1)
        self.b22 = Decoder(same_pad = 1,up_sample=1)
        self.b23 = Decoder(same_pad = 1,up_sample=0)
        self.b24 = Decoder(filters_1x1=3,same_pad = 2,up_sample=0)

    def call(self, inputs):
        l1 = self.b1(inputs)
        l2 = self.b2(l1)
        l3 = self.b12(l2)
        l4 = self.b22(l3)
        l5 = self.b13(l4)
        l6 = self.b23(l5)
        l7 = self.b14(l6)
        l8 = self.b24(l7)

               
        return l8

In [14]:
class Model_comp(tf.keras.Model):
    def __init__(self,filters_1x1=6):
        super(Model_comp, self).__init__()
        self.encoder = Encod_comp()
        self.decoder = Decod_comp()
        self.classifier = Class_out()
    def call(self, inputs):
        l1 = self.encoder(inputs)
        l2 = self.decoder(l1)
        l3 = self.classifier(l1)
        return [l2,l3]

In [23]:
img_data=np.array(img_data)

In [16]:
model_1 =Model_comp()
d = img_data[1:15]
aa=model_1(d)
##bb=model_1(aa[1:15])

In [17]:
steps_per_epoch = 400
model_1 = Model_comp()
model_1.compile(optimizer='adam', loss=['mean_squared_error' , 'categorical_crossentropy'], loss_weights=[1,1],metrics=['accuracy'])
#model_1.compile(loss="mean_squared_error", optimizer='RMSprop')
history = model_1.fit(x=img_data,
    y=[img_data, onehot_encoded],
    batch_size=64,
    epochs=20,
    verbose=1,
    callbacks=None,
    validation_split=0.2,
    shuffle=True)


Train on 34720 samples, validate on 8680 samples
Epoch 1/20
34720/34720 [==============================] - 569s 16ms/sample - loss: 0.7502 - output_1_loss: 0.1895 - output_2_loss: 0.5604 - output_1_accuracy: 0.6387 - output_2_accuracy: 0.7706 - val_loss: 1.0042 - val_output_1_loss: 0.1418 - val_output_2_loss: 0.8626 - val_output_1_accuracy: 0.7796 - val_output_2_accuracy: 0.6975
Epoch 2/20
34720/34720 [==============================] - 556s 16ms/sample - loss: 0.5919 - output_1_loss: 0.1602 - output_2_loss: 0.4316 - output_1_accuracy: 0.6521 - output_2_accuracy: 0.8220 - val_loss: 0.9685 - val_output_1_loss: 0.1204 - val_output_2_loss: 0.8486 - val_output_1_accuracy: 0.7796 - val_output_2_accuracy: 0.6876
Epoch 3/20
34720/34720 [==============================] - 555s 16ms/sample - loss: 0.5672 - output_1_loss: 0.1494 - output_2_loss: 0.4178 - output_1_accuracy: 0.6521 - output_2_accuracy: 0.8257 - val_loss: 0.7317 - val_output_1_loss: 0.1127 - val_output_2_loss: 0.6191 - val_output_1_a

In [18]:
model_1.save('2_3_2021_6am.tf')
model_1.save_weights('2_3_2021_6am_weights.tf')


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: 2_3_2021_6am.tf\assets


In [19]:
np.save('img_data',img_data)
np.save('onehot_encoded',onehot_encoded)

In [24]:
history = model_1.fit(x=img_data,
    y=[img_data, onehot_encoded],
    batch_size=64,
    epochs=10,
    verbose=1,
    callbacks=None,
    validation_split=0.2,
    shuffle=True)

Train on 34720 samples, validate on 8680 samples
Epoch 1/10
34720/34720 [==============================] - 547s 16ms/sample - loss: 0.2080 - output_1_loss: 0.0178 - output_2_loss: 0.1904 - output_1_accuracy: 0.7632 - output_2_accuracy: 0.9410 - val_loss: 0.4424 - val_output_1_loss: 0.0374 - val_output_2_loss: 0.4046 - val_output_1_accuracy: 0.8289 - val_output_2_accuracy: 0.8607
Epoch 2/10
34720/34720 [==============================] - 549s 16ms/sample - loss: 0.1325 - output_1_loss: 0.0173 - output_2_loss: 0.1153 - output_1_accuracy: 0.7557 - output_2_accuracy: 0.9626 - val_loss: 0.4466 - val_output_1_loss: 0.0371 - val_output_2_loss: 0.4092 - val_output_1_accuracy: 0.8317 - val_output_2_accuracy: 0.8797
Epoch 3/10
14144/34720 [===========>..................] - ETA: 5:40:10 - loss: 0.0928 - output_1_loss: 0.0169 - output_2_loss: 0.0758 - output_1_accuracy: 0.7839 - output_2_accuracy: 0.9755

KeyboardInterrupt: 

In [ ]:
model_1.save('2_3_2021_7am.tf')
model_1.save_weights('2_3_2021_7am_weights.tf')


In [ ]:
np.save('img_datap',img_data)
np.save('onehot_encodedp',onehot_encoded)